In [ ]:
import xmltodict
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Configuración de conexión a la base de datos
DB_CONFIG = {
    'host': 'localhost',  # Cambiar si el contenedor de MySQL tiene otro host
    'user': 'root',
    'password': 'tu_contraseña',
    'database': 'etl_db'
}

# Función para leer y procesar el XML
def read_and_process_xml(C:\Users\Estudiante\Documents\feed.xml):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = xmltodict.parse(file.read())

    # Extraer datos de usuarios y propiedades
    users = []
    properties = []

    for item in data['root']['inmueble']:
        # Datos de la propiedad
        properties.append({
            'estado': item['ubicacion']['estado'],
            'ciudad': item['ubicacion']['ciudad'],
            'colonia': item['ubicacion']['colonia'],
            'calle': item['ubicacion']['calle'],
            'numero_exterior': item['ubicacion']['numeroExterior'],
            'tipo_inmueble': item['tipoInmueble'],
            'transaccion': item['transaccion'],
            'precio': float(item['precio']),
            'codigo_proveedor': item['codigoProveedor'],
            'correo_contacto': item['contacto']['correo'],
            'telefono_contacto': item['contacto']['telefono'],
        })

        # Datos del usuario
        users.append({
            'correo_contacto': item['contacto']['correo']
        })

    # Convertir a DataFrames para limpieza
    df_users = pd.DataFrame(users).drop_duplicates()
    df_properties = pd.DataFrame(properties)

    return df_users, df_properties

# Función para cargar datos en MySQL
def load_to_mysql(df_users, df_properties):
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        if connection.is_connected():
            cursor = connection.cursor()

            # Insertar usuarios
            for _, row in df_users.iterrows():
                cursor.execute("""
                    INSERT IGNORE INTO usuarios (correo_contacto)
                    VALUES (%s)
                """, (row['correo_contacto'],))

            # Insertar propiedades
            for _, row in df_properties.iterrows():
                cursor.execute("""
                    INSERT INTO propiedades (
                        estado, ciudad, colonia, calle, numero_exterior,
                        tipo_inmueble, transaccion, precio, codigo_proveedor,
                        correo_contacto, telefono_contacto
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    row['estado'], row['ciudad'], row['colonia'], row['calle'],
                    row['numero_exterior'], row['tipo_inmueble'],
                    row['transaccion'], row['precio'], row['codigo_proveedor'],
                    row['correo_contacto'], row['telefono_contacto']
                ))

            # Confirmar los cambios
            connection.commit()
            print("Datos cargados exitosamente.")
    except Error as e:
        print(f"Error al conectar a MySQL: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# Script principal
if __name__ == "__main__":
    # Ruta al archivo XML
    file_path = 'inmuebles.xml'  # Cambia esto a la ruta correcta

    print("Leyendo y procesando XML...")
    df_users, df_properties = read_and_process_xml(file_path)

    print("Cargando datos a MySQL...")
    load_to_mysql(df_users, df_properties)

    print("ETL completada.")